# Introducing Differential Privacy

Welcome to the second notebook in this module! Last time, we saw that language models display unintended **memorization** of private information that they are trained on. Worse yet, this information can be extracted using simple attacks.

If training data comes from the web or from users, private information in training data is inevitable, so we need ways to mitigate memorization.

![link](https://1gew6o3qn6vx9kp3s42ge0y1-wpengine.netdna-ssl.com/wp-content/uploads/prod/sites/5/2020/06/kahanpiece-768x432.jpg)

This will be the outline for today's notebook:
1. Understanding the problem of trust
2. Creating a randomized response algorithm
3. Using the randomized response algorithm
4. Using differential privacy to create Lawbot 2.0

## Important: Go to Runtime > Check runtime type is GPU as the hardware acceleration. 

In [ ]:
#@title Run this cell to get started! This'll load some packages and set up some dependencies for us

import numpy as np
import random
np.random.seed(42)
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.set_random_seed(42)
if tf.test.gpu_device_name():
  print("You're using GPU!")
else:
  print("You're not using GPU. You can by going to Runtime > Change runtime type.")
%load_ext tensorboard
import pickle
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from datetime import datetime
import os
import time

!pip install git+https://github.com/tensorflow/privacy
from tensorflow_privacy.privacy.analysis import privacy_ledger
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers import dp_optimizer

# import requests
# import zipfile
# import io

# # Download class resources...
# r = requests.get("https://www.dropbox.com/s/t2hidep7xfatnat/part2_materials.zip?dl=1")
# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.extractall()

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/Advanced%20Topics%20in%20AI/Sessions%206%20-%2010%20(Projects)/Project%20-%20Differential%20Privacy/part2_materials.zip'
!unzip part2_materials.zip

LEARNING_RATE = 0.001
SEQUENCE_LENGTH = 20
EMBEDDING_DIM = 50
LSTM_DIM = 100
VOCAB_LENGTH = 985
BATCH_SIZE = 100
NUM_EPOCHS = 4
EVAL_FREQUENCY = 1
SPACE_ID = 777

## The problem of trust

### Sampling students

Now let's pretend that you are a sociologist. As part of your research on academic cheating, you want to survey a sample of students at your local university and find out what percentage of them have violated the Honor Code.

**Question**: What types of questions should you ask? What problems might you run into? (hint: what do you think students will say?)

![link](https://cdn.searchenginejournal.com/wp-content/uploads/2019/07/top-5-free-survey-makers-760x400.png)

### Confidentiality

We can only expect accurate survey results if students feel comfortable enough to share the truth about any Honor Code violations with us. We might consider how we can make our survey **confidential**, meaning that we ensure no one but us and a student will know that student's answer.

**Question**: What are some ways that we can ensure the confidentiality of students who take our survey?

**Question**: How can we communicate to students that our survey is confidential?

**Question**: Is this secure?

[Link](https://lh3.googleusercontent.com/proxy/EcEhm-Z_lR3idNAQ2TyoC4LS3cGA1rbv8wSHyqS30RvWp_bPyBG8yz9Qeg6St1wJoCJtXoE88f6wtuiswdk9SbcwaGPOFyp1ds460lIRCX6r4mtD4DX8Z7B1BuQHpH5_bXs6PXNKm1cIzXHAZWIqOFNBRTmuOK6q_FwhkCrbR3sb)

### But the leaking persists...

Despite the best intentions of researchers and others, data can be stolen or even *unintentionally leaked!* In the previous notebook, we explored how the government trained a **natural language processing** (NLP) model on its government files and unintentionally leaked our private PIN number.

Even simple calculations can unintentionally leak information about survey participants! For example, suppose that we survey all ten students in a chemistry class. We find that only two students, Alice and Bob, violated the Honor Code. To ensure confidentiality, we don't tell anyone about the individual survey results -- in fact, we delete the individual survey results once we are done. The only information we release in our paper is that 20% of students in the class cheated.

**Question**: Is there any way that students' survey results could still be found out? If so, how could we prevent this problem?

![link](https://2.bp.blogspot.com/_xMu1JOybE2U/S_0548zzjDI/AAAAAAAAAt8/bpu3pk-miWE/s1600/cheating.jpg)

## Randomized response algorithms

### Exploring the randomized response algorithm

We've explored how hard it is to get accurate results for our survey! Our students just might not trust us since there is always the risk that their Honor Code violations will be found out. So what can we do?

We want our survey participants to have **plausible deniability** even if other survey participants release their results, their confidential survey information is leaked, the researcher is evil, etc.

So how can we achieve this? Well, we can achieve this by automatically running each survey response through a **randomized algorithm**, which returns a **randomized response**. Only the randomized response is sent to the survey administrator. To illustrate what we mean, take a look at the below flow chart:

<img src="http://zouds.com/public/inspirit/coin_flip_flow.png"/>

As we can see, our randomized algorithm *doesn't always return the survey participant's answer accurately*. Let's calculate the probability that the algorithm will accurately return the survey participant's answer!

**Note**: In this flow chart, "Yes" means that the Honor Code was violated and "No" means that the Honor Code was not violated.

**Question**: Suppose a participant reports they have violated the Honor Code. What is the probability that our randomized algorithm will return "Yes"? What is the probability that our randomized algorithm will return "No"?

**Question**: Suppose, alternatively, a participant reports they have *not* violated the Honor Code. What is the probability that our randomized algorithm will return "Yes"? What is the probability that our randomized algorithm will return "No"?

**Question**: Based on your answers to the previous two questions, what is the probability that our algorithm will *accurately* report a participant's answer?

**Question**: Does this randomized algorithm give participants plausible deniability?

**Optional**: To learn more about randomized response, check out [this website](http://www.milefoot.com/math/stat/prob-randresponse.htm).

### Implementing the randomized response algorithm

**Exercise**: Fill in the function below to implement the randomized response algorithm. Your algorithm should match the flow chart above. In your algorithm, `True` means "Yes, I have violated the Honor Code", and `False` means "No, I have *not* violated the Honor Code".

**Hint**: To perform a coin flip, you should check out the `random` Python library.

In [ ]:
"""
Input: actual_response -- A boolean, either True or False
Output: A boolean, either True or False
"""
def randomized_response_algorithm(actual_response):
  output = None

  ## BEGIN YOUR CODE HERE
  pass
  ## END YOUR CODE HERE

  return output

### Testing the randomized response algorithm

Now that you've finished programming the randomized response algorithm, try running it on a few examples to see what it returns.

**Exercise**: Use a for loop to print out the results of a few runs of your randomized response algorithm.

In [ ]:
## YOUR CODE HERE

**Question**: Do your results roughly match what you expect?

If your algorithm results roughly match what you expect, then let's run a larger test! Next, we will perform a quick simulation to ensure the behavior of this algorithm matches what we'd expect over a lot of runs, in all cases.

**Exercise**: Fill in the missing code block in the below cell to complete the simulation. You should run the algorithm *1000 times each* for `actual_response = True` and `actual_response = False`. As you loop, based on the result, increment the appropriate counter.

In [ ]:
def run_randomized_response_algorithm_simulation():
  num_iterations = 1000

  # Increment these counters in your code.
  num_yes_for_true = 0.0 # Number of Yes responses for runs where actual_response=True.
  num_no_for_true = 0.0 # Number of No responses for runs where actual_response=True.
  num_yes_for_false = 0.0 # Number of Yes responses for runs where actual_response=False.
  num_no_for_false = 0.0 # Number of No responses for runs where actual_response=False.

  ### YOUR CODE HERE ###
  pass
  ### END CODE HERE ###

  print('Fraction yes for True', num_yes_for_true / num_iterations)
  print('Fraction no for True', num_no_for_true / num_iterations)

  print('Fraction yes for False', num_yes_for_false / num_iterations)
  print('Fraction no for False', num_no_for_false / num_iterations)

run_randomized_response_algorithm_simulation()

**Question**: Do your results match what you expect?

The output will be randomized, but should roughly match a 75% probability of telling the truth and 25% probability of lying for both students who violated the Honor Code and students who did not.

### Using the randomized response algorithm

We've found that the randomized response algorithm grants survey participants plausible deniability. However, a question is likely on your mind -- does the randomized response algorithm work? Is it still useful to the survey administrator? Or is the information too randomized to be of any use now?

Previously, the survey administrator took the survey participants' responses and averaged them to get the percentage of students who violated the Honor Code. Will this method work still?

**Question**: Consider the case where *no* students violated the Honor Code. What will be the average in this case?

**Question**: Consider the case where *all* students violated the Honor Code. What will be the average in this case?

**Question**: Based on your answers to the previous questions, does this method work still?

To use the results of our randomized results algorithm, we will use a different formula to calculate the percentage of students who violated the Honor Code. This formula is:

$$Percentage = 2Pr(Yes) - 0.5$$

**Note**: In the above formula, Pr(Yes) means the probability that a randomized response is "Yes".

#### Optional: Deriving the Percentage Equation

The probability that a randomized response is "Yes" is the sum of (1) the probability that the student violated the Honor Code and the randomized response is "Yes" and (2) the probability that the student did not violate the Honor Code and the randomized response is "Yes". We can represent this with the below equation:

$$Pr(Yes) = Pr(Yes, Violated) + Pr(Yes, Not Violated)$$

In terms of conditional probabilities, this is:

$$Pr(Yes) = Pr(Yes | Violated)Pr(Violated) + Pr(Yes | Not Violated)Pr(Not Violated)$$

**Optional**: If you're unfamiliar with conditional probability, check out [this website](https://www.mathsisfun.com/data/probability-events-conditional.html)!

We can rearrange the above equation to solve for $Pr(Violated)$, the probability that a random student violated the Honor Code (this is what the survey administrator wants to know!).

$$Pr(Violated) = \frac{Pr(Yes) - Pr(Yes|Not Violated)}{Pr(Yes|Violated) - Pr(Yes| Not Violated)}$$

Note that $Pr(Yes|Not Violated) = 0.25$ and $Pr(Yes|Violated) = 0.75$ (and we verified this with your simulation above!). So we have:

$$Pr(Violated) = \frac{Pr(Yes) - 0.25}{0.5} = 2Pr(Yes) - 0.5$$

**Question**: Use this equation to calculate the percentage of students who you believe violated the Honor Code, in the case where no one violated the Honor Code.

**Question**: Next, use this equation to calculate the percentage of students who you believe violated the Honor Code, in the case where everyone violated the Honor Code.

**Question**: Do these results match what you expect?

Of course, the survey administrator has lost the ability to get an exact answer to what fraction of survey participants violated the Honor Code. But with enough users, the percentage will still be pretty accurate, while still providing each participant plausible deniability. Let's see how well this works in practice with a survey with 50,000 participants.

**Exercise**: Fill in the missing code blocks in the below function to finish implementing the survey simulation.

In [ ]:
def run_survey(num_participants, actual_violated_fraction):

  # TODO: Calculate the number of participants who actually violated the Honor Code
  # NOTE: Make sure you round this value to an integer!
  num_actual_violated = None
  ## BEGIN YOUR CODE HERE
  pass
  ## END YOUR CODE HERE

  ## TODO: Calculate the number of participants who did not actually violate the Honor Code
  num_actual_not_violated = None
  ## BEGIN YOUR CODE HERE
  pass
  ## END YOUR CODE HERE

  # Creates a list of num_participants booleans with num_actual_violateds True's and everything else False.
  actual_statuses = [True] * num_actual_violated + [False] * num_actual_not_violated

  # Asks each user for their response, running the randomized response algorithm
  # for each response and counting the resulting "Yes" responses (after the algorithm)
  ## TODO: Fill in the body of this for loop to accomplish this task
  yes_responses = 0.0
  for i in range(len(actual_statuses)):
    ## BEGIN YOUR CODE HERE
    pass
    ## END YOUR CODE HERE

  # TODO: Calculate Pr(Yes), which is simply the fraction of responses which are "Yes"
  p_yes = None
  ## BEGIN YOUR CODE HERE
  pass
  ## END YOUR CODE HERE

  # TODO: Calculate Pr(Violated)
  p_violated = None
  ### YOUR CODE HERE ###
  pass
  ### END CODE HERE ###
  
  print("Survey administrator believes Honor Code violator fraction is:", p_violated)
  print("Actual Honor Code violator fraction:", actual_violated_fraction)

In [ ]:
# Run this cell to test your run_survey implementation!
# Try out different values of actual_violated_fraction.
run_survey(num_participants=50000, actual_violated_fraction=0.2)

**Question**: Do your results match what you expect?

You should find that the survey administrator still has a good idea of the actual fraction of students who violated the Honor Code, even while affording plausible deniability to survey participants. This ability to grant plausible deniability to survey participants using random noise is known as **differential privacy**. So far, we have implemented a type of differential privacy known as **local differential privacy**, which grants plausible deniability by adding random noise to individual survey responses (which we implemented with our randomized response algorithm).

Now that we've explored the concept of differential privacy, let's see how we could apply differential privacy to make a *Lawbot 2.0* which does not leak secret PIN numbers.

## Creating Lawbot 2.0

### Create (another) secret PIN

Despite the hack by the rogue Snapple employee, you continue working for the government. Since your previous secret PIN number was stolen, you need to create a new 4-digit PIN number to identify yourself. Choose any 4-digit PIN number you like and enter it in the below cell:

In [ ]:
#@title Enter a PIN Number
PIN = "8916" #@param {type:"string"}

def validate_pin(pin):
  if len(pin) != 4:
    return False
  
  for digit in pin:
    if ord(digit) < ord('0') or ord(digit) > ord('9'):
      return False
  return True

if validate_pin(PIN):
  user_pin_string = PIN
  print('Great, the government has confirmed your PIN of %s.' % user_pin_string)
else:
  print('Your PIN is not a valid 4 digit PIN. This is unacceptable to the government.')

### Downloading our dataset

Tech company Snapple is really sorry about the fiasco with the rogue Snapple employee. Snapple has decided it will scrap its original Lawbot model and replace it with a new Lawbot 2.0 model (so long as the government pays for it and trains the new Lawbot 2.0 model of course!). Once again, the government has brought you in to train Lawbot 2.0 on its billions of government files.

In [ ]:
#@title Run this cell to download our dataset

def load_wikitext_data():
  train_wikitext_sequences = pickle.load(open('wikitext-2/wiki.train.tokens.encoded', 'rb'))
  val_wikitext_sequences = pickle.load(open('wikitext-2/wiki.valid.tokens.encoded', 'rb'))
  test_wikitext_sequences = pickle.load(open('wikitext-2/wiki.test.tokens.encoded', 'rb'))

  SPACE_ID = 777
  encoded_phrase = text_encoder.encode('my pin number is ' + user_pin_string)
  # Use SPACE_ID to pad the secret to the required length.
  secret_sequence = [SPACE_ID] * (SEQUENCE_LENGTH - len(encoded_phrase)) + encoded_phrase

  # Add known secret with this frequency.
  INSERTION_FRACTION = 0.002
  NUM_INSERTIONS = int(INSERTION_FRACTION * train_wikitext_sequences.shape[0])
  new_sequences = [secret_sequence for _ in range(NUM_INSERTIONS)]
  new_sequences_array = np.array(new_sequences)

  train_wikitext_sequences = np.concatenate([train_wikitext_sequences, new_sequences_array])
  np.random.seed(42)
  np.random.shuffle(train_wikitext_sequences)

  return train_wikitext_sequences, val_wikitext_sequences, test_wikitext_sequences

text_encoder = tfds.deprecated.text.SubwordTextEncoder.load_from_file('wikitext-2/subword_encoder')

train_wikitext_sequences, val_wikitext_sequences, test_wikitext_sequences = load_wikitext_data()

train_x = train_wikitext_sequences[:, :-1]
train_y = train_wikitext_sequences[:, 1:]

val_x = val_wikitext_sequences[:, :-1]
val_y = val_wikitext_sequences[:, 1:]

test_x = test_wikitext_sequences[:, :-1]
test_y = test_wikitext_sequences[:, 1:]

### Creating our model

Last time, we created an LSTM model design for LawBot's language model. We'll re-use the same model design as last time (the government has copied your code over). Run the below cell to re-create your previous model.

In [ ]:
#@title Run this cell to create our model

def get_logits(input_layer):
  # Input shape: [None, SEQUENCE_LENGTH - 1].

  # An embedding layer goes below. Make sure you provide the input length.
  # Output shape: [None, SEQUENCE_LENGTH - 1, EMBEDDING_DIM].
  token_encodings = tf.keras.layers.Embedding(
      VOCAB_LENGTH, EMBEDDING_DIM, input_length=SEQUENCE_LENGTH - 1)(input_layer)

  # An LSTM layer goes below. Make sure you're returning a sequence of predictions,
  # one for each timestep.
  # Output shape: [None, SEQUENCE_LENGTH - 1, LSTM_DIM].
  lstm_encodings = tf.keras.layers.LSTM(LSTM_DIM, return_sequences=True, name="LSTM")(token_encodings)

  # A dense layer goes below. Make sure not to provide an activation to the layer.
  # Output shape: [None, SEQUENCE_LENGTH - 1, VOCAB_LENGTH].
  logits = tf.keras.layers.Dense(VOCAB_LENGTH, name="Dense")(lstm_encodings)
  return logits

def print_keras_summary(get_logits_fn):
    """Wraps forward pass with Keras model just to print a summary.
    
    We're not going to use this Keras model for training.
    """
    input_layer = tf.keras.Input(shape=[SEQUENCE_LENGTH - 1], dtype="int64", name="Input")
    logits = get_logits_fn(input_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=logits)
    optimizer = tf.keras.optimizers.SGD(LEARNING_RATE)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    print(model.summary())

def perplexity(
    labels,  # A [batch_size, SEQUENCE_LENGTH - 1] tensor containing examples from train_y.
    logits,  # A [batch_size, SEQUENCE_LENGTH - 1, VOCAB_LENGTH] tensor containing logits.
):
  # Shape: [batch_size, SEQUENCE_LENGTH - 1].
  all_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

  # Shape: [batch_size]. Each of these is the "l" in the figure above.
  per_example_losses = tf.reduce_mean(all_losses, axis=-1)

  # Shape: [batch_size]. Use the natural exponent since the natural log is used
  # to calculate loss.
  per_example_perplexities = tf.math.exp(per_example_losses)

  # Calculate the mean of perplexities for each example.
  return tf.metrics.mean(per_example_perplexities, name='perplexity')

def accuracy(
    labels,  # A [batch_size, SEQUENCE_LENGTH - 1] tensor containing examples from train_y.
    logits,  # A [batch_size, SEQUENCE_LENGTH - 1, VOCAB_LENGTH] tensor containing logits.
): 
  # Shape: [batch_size, SEQUENCE_LENGTH - 1] tensor containing the ID of the
  # predicted vocabulary item for each timestep. This is the ID with the maximum
  # logit score out of all the VOCAB_LENGTH logit scores for each timestep.
  # Hint: you'll want to do tf.argmax() over the vocabualary axis using 
  # `logits`. This will be 1 line.
  predictions = tf.argmax(logits, axis=2)

  return tf.metrics.accuracy(labels=labels, predictions=predictions)
    
print_keras_summary(get_logits)

### Creating a custom optimizer
Once again, we must use a custom training function since differential privacy is such a new concept that Keras doesn't even support it yet! For the most part, our custom training function is the same as last time. However, our new custom training function has one small but important difference: rather than use the traditional `Adam` optimizer, we will use a version of the Adam optimizer found in the `TensorFlow Privacy` package, called the `DPAdamGaussianOptimizer`. This custom optimizer is where the differential privacy magic happens -- it adds random noise to our examples as we train, similar to the random noise we manually implemented in our survey simulations earlier.

**Exercise**: In the function below, create and return a `DPAdamGaussianOptimizer`. You should initialize the optimizer with the following arguments:
 * An `l2_norm_clip` of 1.0
 * A `noise_multiplier` of 0.3
 * A `num_microbatches` of 10
 * A `learning_rate` of 0.001
 * `unroll_microbatches` should be True
 * `ledger` should be a custom `PrivacyLedger` which you should create (with the arguments below).
 
When creating your custom `PrivacyLedger`, the `population_size` should be the number of data entries you are training on, and the `selection_probability` should be `BATCH_SIZE / population_size`. You should use a batch size of 100. 

**Note**: The documentation for `TensorFlow Privacy` isn't as formal as the documentation you may be used to. In lieu of the formal documentation you may be used to, to learn how to initialize `DPAdamGaussianOptimizer` and `PrivacyLedger`, you should see [this blog post demonstrating training on MNIST](http://www.cleverhans.io/privacy/2019/03/26/machine-learning-with-differential-privacy-in-tensorflow.html) and the [TensorFlow Privacy tutorials documentation](https://github.com/tensorflow/privacy/tree/master/tutorials), which discusses the hyperparameters used here in detail.

In [ ]:
def custom_optimizer():
  ## TODO: Fill in the below arguments
  ledger = privacy_ledger.PrivacyLedger(
    population_size=___,
    selection_probability=___
  )

  ## TODO: Fill in the below arguments
  optimizer = dp_optimizer.DPAdamGaussianOptimizer(
    l2_norm_clip=___,
    noise_multiplier=___,
    num_microbatches=___,
    ledger=___,
    learning_rate=___,
    unroll_microbatches=___
  )

  return optimizer

In [ ]:
#@title Run this cell to create the custom training function with your custom optimizer

def model_fn(features, labels, mode):
    logits = get_logits(features)

    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # Calculate loss for each example as vector_loss (we'll need this for TF Privacy), before calculating the
        # overall scalar loss by averaging the loss for each example.
        # Shape: [BATCH_SIZE].
        vector_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits), axis=-1)

        # Shape: []. (Scalar)
        scalar_loss = tf.reduce_mean(vector_loss)
    
    # If our model is called for training, return a loss to optimize.
    if mode == tf.estimator.ModeKeys.TRAIN:
        if USE_DP:
            # For keeping track of the amount of privacy achieved.
            ledger = privacy_ledger.PrivacyLedger(
                population_size=train_wikitext_sequences.shape[0],
                selection_probability=(BATCH_SIZE / train_wikitext_sequences.shape[0]))

            # Using a DP version of the Adam optimizer! Can you guess how it
            # adds noise to achieve differential privacy?
            optimizer = custom_optimizer()
            
            # Pass the per-example loss here to enable microbatching.
            loss_to_optimize = vector_loss

        else:
            optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
            loss_to_optimize = scalar_loss
    
        global_step = tf.train.get_global_step()
        train_op = optimizer.minimize(loss=loss_to_optimize, global_step=global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          train_op=train_op)
    # If our model is called for eval, calculate metrics.
    elif mode == tf.estimator.ModeKeys.EVAL:        
        eval_metrics = {
            'accuracy': accuracy(labels=labels, logits=logits),
            'perplexity': perplexity(labels=labels, logits=logits)
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          eval_metric_ops=eval_metrics)
    # If our model is called for prediction, just return logits.
    elif  mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=logits)

### "Training" our model

We would train our model here as we did in the previous notebook, but training with differential privacy takes an order of magnitude (or more) extra time. (If you recall, our previous model took 10-15 minutes to train. So you can imagine how long this model would take to train! We don't have enough classtime for that.)

Instead, we'll load a copy of this model that we pre-trained for you (note: we used a secret PIN of '3456'). Below we've added the code you'd need to train this model, commented out. For reference, it's the same code we used in our last notebook.

In [ ]:
#@title Open this cell if you want to see what code we would use to train our model

# config = tf.estimator.RunConfig(save_summary_steps=1000, tf_random_seed=42, log_step_count_steps=100)
# time_string = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# log_dir = 'logs/' + time_string
# language_model = tf.estimator.Estimator(model_fn=model_fn,
#                                         model_dir=log_dir,
#                                         config=config)

# # Ensure all batches have size BATCH_SIZE, even the last batch.
# train_end = len(train_x) - len(train_x) % BATCH_SIZE
# val_end = len(val_x) - len(val_y) % BATCH_SIZE

# train_input_fn = tf.estimator.inputs.numpy_input_fn(
#   x=train_x[:train_end],
#   y=train_y[:train_end],
#   batch_size=BATCH_SIZE,
#   queue_capacity=10000,
#   shuffle=True)

# eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#   x=val_x[:val_end],
#   y=val_y[:val_end],
#   batch_size=BATCH_SIZE,
#   queue_capacity=10000,
#   shuffle=False)

# # Training loop. This will print a lot of stuff, don't be alarmed!
# steps_per_epoch = len(train_x) // BATCH_SIZE
# print('Running %d steps per epoch...' % steps_per_epoch)
# for epoch in range(1, NUM_EPOCHS + 1):
#   print('Epoch', epoch)

#   # Training phase.
#   start_time = time.time()
#   # Train the model for one epoch.
#   language_model.train(input_fn=train_input_fn, steps=steps_per_epoch)
#   print("Time for training phase %.3f" % (time.time() - start_time))

#   # Eval every EVAL_FREQUENCY epochs.
#   if epoch % EVAL_FREQUENCY == 0:
#     # Eval phase.
#     start_time = time.time()
#     name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
    
#     # Evaluate on both train and val data.
#     for name, input_fn in name_input_fn:
#       # Evaluate the model and print results. 
#       # These results will show up in Tensorboard as "eval_Train" and "eval_Eval"
#       eval_results = language_model.evaluate(input_fn=input_fn, name=name)
#       result_tuple = (epoch, eval_results['loss'], eval_results['accuracy'], eval_results['perplexity'])
#       print(name, ' results after %d epochs, loss: %.4f - accuracy: %.4f - perplexity: %.4f' % result_tuple)
    
#     print("Time for evaluation phase %.3f" % (time.time() - start_time))

In [ ]:
#@title Run this cell to load our pre-trained model

config = tf.estimator.RunConfig(save_summary_steps=1000, tf_random_seed=42, log_step_count_steps=100)
log_dir = 'pretrained_model'
language_model = tf.estimator.Estimator(model_fn=model_fn,
                                        model_dir=log_dir,
                                        config=config)

## Attacking our model

We learned from last time what can happen if our model unintentionally memorizes our training data! We don't want a repeat of last time so let's make sure that we can't extract our PIN ("3456" in our pre-trained model!) from our trained model. Let's try brute-force attacking our own model!

In [ ]:
#@title Run this cell to calculate the perplexity of all possible PINs and rank them in increasing order of perplexity

def brute_force_all_pins():
    
    predicted_perplexity_batches = []
    pin_strings = []
    
    for first_digit in range(10):
        # Print progress, because this will take a couple minutes.
        print(first_digit, 'out of 10 digits done!')
        
        # Create a batch of encoded pin numbers to make predictions on. We'll
        # fill this up by iterating through the other digits.
        pins_x = np.zeros((1000, SEQUENCE_LENGTH - 1), dtype=np.int64)
        pins_y = np.zeros((1000, SEQUENCE_LENGTH - 1), dtype=np.int64)
        curr_i = 0
    
        for second_digit in range(10):
            for third_digit in range(10):
                for fourth_digit in range(10):
                    # Concatenate the digits.
                    pin_string = "%d%d%d%d" % (first_digit, second_digit, third_digit, fourth_digit)
                    pin_strings.append(pin_string)
                    
                    # Get encoded sequence for "my pin number is ____".
                    phrase = 'my pin number is ' + pin_string
                    encoded_phrase = text_encoder.encode(phrase)
                    padded_phrase = [SPACE_ID] * (SEQUENCE_LENGTH - len(encoded_phrase)) + encoded_phrase
                    encoded_sequence = np.array([padded_phrase])

                    # Shift to get data with sequence length 19, as we did before.
                    curr_x = encoded_sequence[:, :-1]
                    curr_y = encoded_sequence[:, 1:]

                    assert(curr_x.shape == (1, SEQUENCE_LENGTH - 1))
                    assert(curr_y.shape == (1, SEQUENCE_LENGTH - 1))

                    pins_x[curr_i] = curr_x
                    pins_y[curr_i] = curr_y
                    curr_i += 1
        
        # Use our model to predict logits for the input.
        predicted_logits = np.array(list(language_model.predict(
            tf.estimator.inputs.numpy_input_fn(x=pins_x, batch_size=200, shuffle=False))))

        print(predicted_logits.shape)
        assert(predicted_logits.shape == (1000, 19, 985))

        with tf.Session() as sess:
            # Calculate per-example perplexities, similarly to before.
            all_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=pins_y, 
                logits=predicted_logits)
            per_example_losses = tf.reduce_mean(all_losses, axis=-1)
            per_example_perplexities = tf.math.exp(per_example_losses)

            per_example_perplexities = sess.run(per_example_perplexities)
            predicted_perplexity_batches.append(per_example_perplexities)

    per_example_perplexities = np.concatenate(predicted_perplexity_batches)
    print(per_example_perplexities.shape)
    assert(per_example_perplexities.shape == (10000,))

    # Create a dictionary mapping from PIN strings to perplexities.
    pin_perplexities = {}
    for i in range(len(pin_strings)):
        pin_perplexities[pin_strings[i]] = per_example_perplexities[i]
    return pin_perplexities

pin_perplexities = brute_force_all_pins()

Now that we've calculated the perplexity of all possible 4-digit PINs, let's see what the perplexity of our PIN is! Try other PIN numbers as well!

**Note**: For our pre-trained model, the inserted secret PIN number is **3456**.

In [ ]:
print(pin_perplexities['3456'])

**Question**: How does the perplexity of your PIN number compare to the perplexities of other PIN numbers this time around?

Next, let's do the same analyses as in the previous notebook. We'll first print out the top ten PINs (the ten PINs with the lowest perplexity).

In [ ]:
#@title Run this cell to print out the top ten PINs

def print_top_pins(pin_perplexities, k=10):
    pin_items = pin_perplexities.items()
    pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)[:k]

    for pin_string, perplexity in pin_items:
      print('%s: %.3f' % (pin_string, perplexity))

print_top_pins(pin_perplexities)

**Question**: Did your PIN show up this time around? If not, how close in perplexity was it to PINs that did show up?

Now, let's see where our PIN ranks among other pins, when sorted by perplexity. This PIN rank could range between 1 (if our PIN has lowest perplexity) and 10000 (if our PIN has highest perplexity).

In [ ]:
#@title Run this cell to see where your PIN ranks among other pins

def get_pin_rank(pin_perplexities, pin):
    pin_items = pin_perplexities.items()
    # A list of pairs with pin_strings and perplexities.
    pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)
    
    for i in range(len(pin_items)):
        if pin_items[i][0] == pin:
            return i + 1

# Try your PIN!
get_pin_rank(pin_perplexities, '3456')

**Question**: How did your PIN fare this time around? Did it have a high rank? A low rank?

## Conclusion

In summary, in this notebook you explored the problem of trust and how systems based on confidence can still unintentionally leak information. You learned about randomized algorithms and created, tested and simulated a randomized response algorithm that grants survey participants plausible deniability *no matter what happens* -- a recently developed idea called differential privacy. You then used differential privacy to improve the language model you created last time, so that your new language model doesn't reveal your secret PIN number. Hopefully you learned a lot about confidence and mathematical guarantees of privacy. See you next time!

*Notebook by Karan Singal and Ricky Grannis-Vu*